In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import re
from os import listdir
from os.path import isfile, join
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import classification_report

     |████████████████████████████████| 115 kB 1.2 MB/s 


# 1. Sentiment analysis

Using the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), we want to do a regression model that predict the ratings are on a 1-10 scale. You have an example train and test set in the `dataset` folder.

### 1.1 Regression Model

Use a feedforward neural network and NLP techniques we've seen up to now to train the best model you can on this dataset

### 1.2 RNN model

Train a RNN to do the sentiment analysis regression. The RNN should consist simply of an embedding layer (to make word IDs into word vectors) a recurrent blocks (GRU or LSTM) feeding into an output layer.

In [2]:
#Work in progress

In [3]:
def get_review(row):
    path = row['path']
    file = row['file']
    with open(f'{path}/{file}') as f:
        file_content = f.read()
        soup = BeautifulSoup(file_content)
        text = soup.get_text()
    return text

def get_id(row):
    file = row['file']
    print(file)
    split_str = re.split('_|.t',file)
    return split_str[0]

def get_rating(row):
    file = row['file']
    split_str = re.split('_|.t',file)
    return split_str[1]

In [4]:
train_pos = []
train_neg = []
test_pos = []
test_neg = []

path = '../input/movies/aclImdb'
data_type = ['train', 'test']
sentiments = ['pos', 'neg']
d_tmp = {}


for dtype in data_type:
    for s in sentiments:
        file_path = f'{path}/{dtype}/{s}'
        files = [f for f in listdir(file_path) if isfile(join(file_path, f))]
        key = f'{dtype}_{s}'
        d_tmp[key+'_file'] = files
        d_tmp[key+'_path'] = [file_path]*len(files)
        if s == 'pos':
            d_tmp[key+'_pos'] = [1]*len(files) 
        elif s =='neg':
            d_tmp[key+'_pos'] = [0]*len(files) 
            
train_pos = pd.DataFrame({'path': d_tmp['train_pos_path'],'file':d_tmp['train_pos_file'],'pos':d_tmp['train_pos_pos'] })
train_neg = pd.DataFrame({'path': d_tmp['train_neg_path'],'file':d_tmp['train_neg_file'],'pos':d_tmp['train_neg_pos']})
test_pos = pd.DataFrame({'path': d_tmp['test_pos_path'],'file':d_tmp['test_pos_file'],'pos':d_tmp['test_pos_pos']})
test_neg = pd.DataFrame({'path': d_tmp['test_neg_path'],'file':d_tmp['test_neg_file'],'pos':d_tmp['test_neg_pos']})

In [5]:
test_pos

,path,file,pos
0,../input/movies/aclImdb/test/pos,589_10.txt,1
1,../input/movies/aclImdb/test/pos,6451_9.txt,1
2,../input/movies/aclImdb/test/pos,2750_8.txt,1
3,../input/movies/aclImdb/test/pos,5746_10.txt,1
4,../input/movies/aclImdb/test/pos,2658_10.txt,1
...,...,...,...
12495,../input/movies/aclImdb/test/pos,10090_10.txt,1
12496,../input/movies/aclImdb/test/pos,4019_7.txt,1
12497,../input/movies/aclImdb/test/pos,12380_7.txt,1
12498,../input/movies/aclImdb/test/pos,12137_8.txt,1


In [6]:
dfs = [train_pos, train_neg, test_pos, test_neg]

for df in dfs:
    df['id'] = df.apply(get_id, axis = 1)
    df['id'] = df['id'].astype(int)
    df['rating'] = df.apply(get_rating, axis = 1)
    df['rating'] = df['rating'].astype(int)
    df['review'] = df.apply(get_review, axis = 1)

2714_10.txt
589_10.txt
2211_8.txt
2658_10.txt
8929_8.txt
5651_10.txt
6648_7.txt
9503_7.txt
8951_7.txt
3092_10.txt
8361_8.txt
11853_7.txt
11064_7.txt
9538_10.txt
8994_8.txt
12337_7.txt
9299_8.txt
5949_10.txt
5122_9.txt
3900_10.txt
2476_10.txt
1233_7.txt
9250_8.txt
984_7.txt
7752_8.txt
5074_10.txt
2285_10.txt
963_8.txt
7984_10.txt
5195_7.txt
1794_7.txt
5186_10.txt
9692_10.txt
2666_10.txt
411_10.txt
824_8.txt
10533_10.txt
2692_8.txt
1393_7.txt
2857_9.txt
8133_7.txt
1085_7.txt
10402_10.txt
5531_10.txt
915_10.txt
12267_8.txt
3411_9.txt
2567_9.txt
3749_9.txt
5540_10.txt
12272_7.txt
2950_10.txt
9022_10.txt
11407_7.txt
9797_7.txt
6797_8.txt
4153_10.txt
7270_8.txt
289_10.txt
5725_9.txt
5729_9.txt
11098_10.txt
7997_10.txt
8719_10.txt
1001_8.txt
9402_7.txt
7801_9.txt
2484_10.txt
10850_10.txt
1095_9.txt
2985_8.txt
6309_8.txt
4883_10.txt
2430_10.txt
12177_7.txt
2965_9.txt
8431_9.txt
11525_8.txt
705_10.txt
1433_10.txt
3765_10.txt
5607_7.txt
97_9.txt
1405_7.txt
2984_8.txt
630_10.txt
11427_10.txt
8992

In [7]:
def preprocessing(series):
    vectorizer = TfidfVectorizer(stop_words='english', min_df=2, max_df=0.96)
    X = vectorizer.fit_transform(series)
    scaler = MaxAbsScaler()
    X = scaler.fit_transform(X)
    svd = TruncatedSVD(n_components=2000, n_iter=7, random_state=42)
    X = svd.fit_transform(X)

    return X
    

In [8]:
train = pd.concat([train_pos, train_neg]).sample(frac = 1).reset_index(drop =True)


In [9]:
X_train_tot = preprocessing(train['review'])

In [10]:
X_train_ncols = X_train_tot.shape[1]
X_train_ncols 

2000

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    25000 non-null  object
 1   file    25000 non-null  object
 2   pos     25000 non-null  int64 
 3   id      25000 non-null  int64 
 4   rating  25000 non-null  int64 
 5   review  25000 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.1+ MB


In [12]:
y_train_tot = train['rating']


In [13]:
X_valid, X_train = X_train_tot[:7500, :], X_train_tot[7500:, :]
y_valid, y_train = y_train_tot[:7500], y_train_tot[7500:]

In [14]:
train['rating'].unique()

array([ 1,  7, 10,  9,  8,  2,  4,  3])

In [15]:
model = keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(X_train_ncols,)),
        layers.Dense(300, activation="relu"),
        layers.Dense(100, activation="relu"),
        layers.Dense(11, activation = 'softmax'),
    ]
)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               600300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 11)                1111      
Total params: 631,511
Trainable params: 631,511
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(X_train, y_train, epochs = 35, validation_data= (X_valid, y_valid))

Epoch 1/35
547/547 [==============================] - 3s 4ms/step - loss: 1.8754 - accuracy: 0.3072 - val_loss: 1.4945 - val_accuracy: 0.4192
Epoch 2/35
547/547 [==============================] - 1s 3ms/step - loss: 1.2346 - accuracy: 0.5338 - val_loss: 1.5590 - val_accuracy: 0.4048
Epoch 3/35
547/547 [==============================] - 1s 3ms/step - loss: 0.8837 - accuracy: 0.6889 - val_loss: 1.7739 - val_accuracy: 0.3883
Epoch 4/35
547/547 [==============================] - 1s 3ms/step - loss: 0.4639 - accuracy: 0.8676 - val_loss: 2.2388 - val_accuracy: 0.3744
Epoch 5/35
547/547 [==============================] - 1s 3ms/step - loss: 0.1450 - accuracy: 0.9756 - val_loss: 2.8793 - val_accuracy: 0.3653
Epoch 6/35
547/547 [==============================] - 1s 3ms/step - loss: 0.0326 - accuracy: 0.9987 - val_loss: 3.3969 - val_accuracy: 0.3653
Epoch 7/35
547/547 [==============================] - 2s 3ms/step - loss: 0.0085 - accuracy: 0.9996 - val_loss: 3.7201 - val_accuracy: 0.3620
Epoch 

In [18]:
test = pd.concat([test_pos, test_neg]).sample(frac=1).reset_index(drop =True)
test

,path,file,pos,id,rating,review
0,../input/movies/aclImdb/test/pos,7232_8.txt,1,7232,8,"Once upon a time, Troma, the company that brou..."
1,../input/movies/aclImdb/test/pos,1496_10.txt,1,1496,10,"Mr Bean was great fun, i loved it, every episo..."
2,../input/movies/aclImdb/test/pos,10362_7.txt,1,10362,7,I liked this movie. Many people refer to it as...
3,../input/movies/aclImdb/test/pos,2231_7.txt,1,2231,7,There's a good running bit about the price tag...
4,../input/movies/aclImdb/test/pos,7894_9.txt,1,7894,9,I was surprised by how emotionally invested I ...
...,...,...,...,...,...,...
24995,../input/movies/aclImdb/test/pos,7759_9.txt,1,7759,9,"Good animation, nice character design, and a l..."
24996,../input/movies/aclImdb/test/pos,9682_9.txt,1,9682,9,"""Family Guy"" is probably the most ballsy sitco..."
24997,../input/movies/aclImdb/test/pos,9972_9.txt,1,9972,9,The inherent problem with any staging of 'The ...
24998,../input/movies/aclImdb/test/neg,11539_3.txt,0,11539,3,"ExCUSE me, but my tongue was TOO in my cheek w..."


In [19]:
X_test = preprocessing(test['review'])

In [20]:
test

,path,file,pos,id,rating,review
0,../input/movies/aclImdb/test/pos,7232_8.txt,1,7232,8,"Once upon a time, Troma, the company that brou..."
1,../input/movies/aclImdb/test/pos,1496_10.txt,1,1496,10,"Mr Bean was great fun, i loved it, every episo..."
2,../input/movies/aclImdb/test/pos,10362_7.txt,1,10362,7,I liked this movie. Many people refer to it as...
3,../input/movies/aclImdb/test/pos,2231_7.txt,1,2231,7,There's a good running bit about the price tag...
4,../input/movies/aclImdb/test/pos,7894_9.txt,1,7894,9,I was surprised by how emotionally invested I ...
...,...,...,...,...,...,...
24995,../input/movies/aclImdb/test/pos,7759_9.txt,1,7759,9,"Good animation, nice character design, and a l..."
24996,../input/movies/aclImdb/test/pos,9682_9.txt,1,9682,9,"""Family Guy"" is probably the most ballsy sitco..."
24997,../input/movies/aclImdb/test/pos,9972_9.txt,1,9972,9,The inherent problem with any staging of 'The ...
24998,../input/movies/aclImdb/test/neg,11539_3.txt,0,11539,3,"ExCUSE me, but my tongue was TOO in my cheek w..."


In [21]:
y_true = test['rating']

In [22]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

In [23]:
y_pred

array([ 2, 10,  4, ...,  9,  9,  7])

In [24]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.33      0.37      0.35      5022
           2       0.12      0.11      0.12      2302
           3       0.14      0.12      0.13      2541
           4       0.14      0.15      0.15      2635
           7       0.11      0.12      0.11      2307
           8       0.14      0.15      0.15      2850
           9       0.14      0.13      0.14      2344
          10       0.35      0.33      0.34      4999

    accuracy                           0.22     25000
   macro avg       0.18      0.18      0.18     25000
weighted avg       0.22      0.22      0.22     25000



In [25]:
y_train_sent_tot = train['pos']
y_valid_sent, y_train_sent = y_train_sent_tot[:7500], y_train_sent_tot[7500:]

In [26]:
y_true = test['pos']
y_true

0        1
1        1
2        1
3        1
4        1
        ..
24995    1
24996    1
24997    1
24998    0
24999    1
Name: pos, Length: 25000, dtype: int64

In [27]:
print(classification_report(y_true, y_sent_pred.flatten()))

NameError: name 'y_sent_pred' is not defined

In [28]:
results = model_sentiment.evaluate(X_test, y_true, batch_size=128)

NameError: name 'model_sentiment' is not defined

In [29]:
results

NameError: name 'results' is not defined

In [30]:
#Work in progress
#2.2 Source Hands-on Marchine Learning with Scikit-Learn, Keras & TensorFlow

import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info = True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9HMEFH/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9HMEFH/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [31]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value = b"<pad>"), y_batch



In [32]:
from collections import Counter 
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [33]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [34]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)
table.lookup(tf.constant(b"This movie was faaaaaantastic".split()))

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([   22,    12,    11, 10053])>

In [35]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [36]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape = [None]),
    keras.layers.GRU(128, return_sequences = True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history = model.fit(train_set, epochs = 5)

Epoch 1/5
782/782 [==============================] - 23s 24ms/step - loss: 0.6755 - accuracy: 0.5461
Epoch 2/5
782/782 [==============================] - 19s 24ms/step - loss: 0.4711 - accuracy: 0.7789
Epoch 3/5
782/782 [==============================] - 19s 24ms/step - loss: 0.3277 - accuracy: 0.8678
Epoch 4/5
782/782 [==============================] - 20s 25ms/step - loss: 0.2026 - accuracy: 0.9270
Epoch 5/5
782/782 [==============================] - 19s 24ms/step - loss: 0.1556 - accuracy: 0.9456


2. (evil) XOR Problem
 parity
Train an LSTM to solve the XOR problem: that is, given a sequence of bits, determine its parity. The LSTM should consume the sequence, one bit at a time, and then output the correct answer at the sequence’s end. Test the two approaches below:

2.1
Generate a dataset of random <=100,000 binary strings of equal length <= 50. Train the LSTM; what is the maximum length you can train up to with precisison?

2.2
Generate a dataset of random <=200,000 binary strings, where the length of each string is independently and randomly chosen between 1 and 50. Train the LSTM. Does it succeed? What explains the difference?

In [37]:
#Work in progress
#Source: https://vitez.me/lstm-xor
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input, LSTM
from tensorflow.keras.models import Sequential
import numpy as np
import random


SEQ_LEN = 50
COUNT = 100000

bin_pair = lambda x: [x, not(x)]
training = np.array([[bin_pair(random.choice([0, 1])) for _ in range(SEQ_LEN)] for _ in range(COUNT)])
target = np.array([[bin_pair(x) for x in np.cumsum(example[:,0]) % 2] for example in training])

print('shape check:', training.shape, '=', target.shape)

model = Sequential()
model.add(Input(shape=(SEQ_LEN, 2), dtype='float32'))
model.add(LSTM(1, return_sequences=True))
model.add(Dense(2, activation='softmax'))

predictions = model.predict(training)
i = random.randint(0, COUNT)
chance = predictions[i,-1,0]
print('randomly selected sequence:', training[i,:,0])
print('prediction:', int(chance > 0.5))
print('confidence: {:0.2f}%'.format((chance if chance > 0.5 else 1 - chance) * 100))
print('actual:', np.sum(training[i,:,0]) % 2)

shape check: (100000, 50, 2) = (100000, 50, 2)
randomly selected sequence: [1 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1]
prediction: 0
confidence: 50.90%
actual: 0
